In [2]:
## UV decomposition using vectorized operations and Alternating least Squares method

import numpy as np
import pandas as pd

In [57]:
M = np.array([[1,2,3,4,5],[np.nan,2,np.nan,4,1],[5,np.nan,np.nan,5,np.nan],[np.nan,2,3,4,np.nan]])
M
#np.eye(10)

array([[  1.,   2.,   3.,   4.,   5.],
       [ nan,   2.,  nan,   4.,   1.],
       [  5.,  nan,  nan,   5.,  nan],
       [ nan,   2.,   3.,   4.,  nan]])

## Alternating Least Squares Algorithm

_Objective_: Express the matrix M as a product of U and V. Where _M_ is a _mXn_ matrix, _U_ is a _mXd_ matrix and _V_ is a _dXn_ matrix, where _d_ is the number of latent factors.


Mathematically, let
$$M=\left[ \begin{array}{cccccc} r_{11} & r_{12} & . & . & r_{1n} \\ r_{21} & r_{22} & . & . & r_{2n} \\ r_{31} & r_{32} & . & . & r_{3n} \\ . & . & . & . & . \\  . & . & . & . & . \\ r_{m1} & r_{m2} & . & . & r_{mn} \end{array} \right],U=\left[ \begin{array}{ccc} u_{11} & . & u_{1d} \\ u_{21} & . & u_{2d} \\  u_{31} & . & u_{3d} \\ . & . & . \\ .& . & . \\ u_{m1} & . & u_{md} \end{array} \right], V=\left[ \begin{array}{ccccc} v_{11} & v_{12} & . & . & v_{1n} \\ v_{21} & v_{22} & . & . & v_{2n} \\ . & . & . & . & . \\v_{d1} & v_{d2} & . & . & v_{dn}\end{array} \right]
$$

Then M can be expressed as the matrix product of U and V, as shown below: 
$$\left[ \begin{array}{cccccc} r_{11} & r_{12} & . & . & r_{1n} \\ r_{21} & r_{22} & . & . & r_{2n} \\ r_{31} & r_{32} & . & . & r_{3n} \\ . & . & . & . & . \\  . & . & . & . & . \\ r_{m1} & r_{m2} & . & . & r_{mn} \end{array} \right] = 
\left[ \begin{array}{ccc} u_{11} & . & u_{1d} \\ u_{21} & . & u_{2d} \\  u_{31} & . & u_{3d} \\ . & . & . \\ .& . & . \\ u_{m1} & . & u_{md} \end{array} \right] . \left[ \begin{array}{ccccc} v_{11} & v_{12} & . & . & v_{1n} \\ v_{21} & v_{22} & . & . & v_{2n} \\ . & . & . & . & . \\v_{d1} & v_{d2} & . & . & v_{dn}\end{array} \right] $$


In recommender systems, the matrix M is a sparse matrix with many unknown entries. An example of such sparse matrix is shown below:

$$M=\left[ \begin{array}{cccccc}  & r_{12} & . & . & r_{1n} \\ r_{21} &   & . & . & r_{2n} \\ r_{31} &   & . & . &   \\ . & . & . & . & . \\  . & . & . & . & . \\ r_{m1} & r_{m2} & . & . &   \end{array} \right]$$

Our main objective is to estimate the U and V matrices considering only the available data in M. One method to estimate the U and V matrices is _Alternating Least Squares (ALS)_ method. The algorithm for ALS is given below:

1. Initialize U and V matrices to random real numbers
2. Let i represent row number in U and j represent the column number in V
3. To estimate the row $U_i$, perform the following:

   3a. Get the list of all columns in $M_i$ row, where we have available values. Call these locations as C

   3b. Let $V_C = V[:,C]$, where $V[:,C]$ is the list of all columns in V, corresponding to the column numbers present in C
   
4. Estimate new value of $U_i$ as:
$$U^{new}_{i} = {(\lambda_1.I + V_C . V^T_C)}^{-1} (V_C . M^T_i)$$ where $I$ is an indentity matrix of size equal to d

5. Update $U_i = {(U^{new}_{i})}^T$

6. To estimate the column $V_j$, perform the following:
   
   6a. Get the list of all rows in $M_j$ column, where we have available values. Call these locations as R
   
   6b. Let $U_R = U[R,:]$, where $U[R,:]$ is the list of all rows in U, corresponding to the row numbers present in R

7. Estimate new value of $V_j$ as:
$$V^{new}_{j} = {(\lambda_2.I + U^T_R . U_R)}^{-1} (U^T_R . M_j)$$ where $I$ is an indentity matrix of size equal to d


8. Update $V_j = V^{new}_j$

9. Get the U.V 

10. Subtract the available values of R from the corresponding values of U.V. Square the resulting values, sum them and take the square root to obtain RMSE (Root Mean Squared Error)

11. Repeat steps 3 to 10, until desired number of iterations or until RMSE is stablized

Note: The $\lambda_1$ and $\lambda_2$ are regularization parameters. We have to tune these parameters to optimal values. 

In [5]:
#Get the error
def get_RMSE_error(M,U,V):
    return np.sqrt(np.nanmean(np.square(M - np.dot(U,V))))

In [135]:
def compute_U_row(i,M,V,d,lambda1):
    '''
     i = desired row number in U, which needs to be computed
     M = Utility matrix of size mXn
     V = V component of size dXn
     lambda1 = reg factor
    '''
    num_of_rows, num_of_cols = M.shape
    #print "num_of_users = {} and num_of_items = {}".format(num_of_rows, num_of_cols)
    
    #np.where will return a tuple.
    C = np.where(~np.isnan(M[i,]))[0]
    
    V_C = V[:,C].copy()
    M_i = M[i,:].copy()
    M_i = M_i[~np.isnan(M_i)]
    #print "V_C={}".format(V_C)
    #print "M_i={}".format(M_i)
    #V_C = np.nan_to_num(V_C)
    #print np.linalg.inv(np.eye(d) * lambda1 + np.dot(V_C,V_C.T)) 
    #print np.dot(V_C,M_i.T)
    #print np.dot(np.linalg.inv(np.eye(d) * lambda1 + np.dot(V_C,V_C.T)),np.dot(V_C,M_i.T).T)
    return np.dot(np.linalg.inv(np.eye(d) * lambda1 + np.dot(V_C,V_C.T)),np.dot(V_C,M_i.T).T)


def compute_V_col(j,M,U,d,lambda2):
    '''
     j = desired column number in V, which needs to be computed
     M = Utility matrix of size mXn
     U = U component of size mXd
     lambda2 = reg factor
    '''
    num_of_rows, num_of_cols = M.shape
    #print "num_of_users = {} and num_of_items = {}".format(num_of_rows, num_of_cols)
    
    #np.where will return a tuple.
    R = np.where(~np.isnan(M[:,j]))[0]
    
    U_R = U[R,:].copy()
    M_j = M[:,j].copy()
    M_j = M_j[~np.isnan(M_j)]
    #print "V_C={}".format(V_C)
    #print "M_i={}".format(M_i)
    #V_C = np.nan_to_num(V_C)
    #print np.linalg.inv(np.eye(d) * lambda1 + np.dot(V_C,V_C.T)) 
    #print np.dot(V_C,M_i.T)
    #print "U_R = {}".format(U_R)
    #print "M_j = {}".format(M_j)
    #print np.linalg.inv(np.eye(d) * lambda2+ np.dot(U_R.T,U_R))
    #print np.dot(U_R.T,M_j).T
    #print np.dot(np.linalg.inv(np.eye(d) * lambda2 + np.dot(U_R.T,U_R)),np.dot(U_R.T,M_j).T)
    return np.dot(np.linalg.inv(np.eye(d) * lambda2 + np.dot(U_R.T,U_R)),np.dot(U_R.T,M_j).T).T

In [136]:
i = 1
seed = 10
d = 2
np.random.seed(seed)
U = np.zeros((M.shape[0],d),dtype=np.float)+np.random.rand(M.shape[0],d)
#U = np.zeros((M_rows,factors),dtype=np.float)+.2
print U
np.random.seed(seed)
V = np.zeros((d,M.shape[1]),dtype=np.float)+np.random.rand(d,M.shape[1])
print V

print "error 0"
print get_error(M,U,V)

lambda1 = 1
compute_U_row(i,M,V,d,lambda1)
U[i,] = compute_U_row(i,M,V,d,lambda1)
print U

print "error 1"
print get_error(M,U,V)

i=2
compute_U_row(i,M,V,d,lambda1)
U[i,] = compute_U_row(i,M,V,d,lambda1)
print U

print "error 2"
print get_error(M,U,V)

i=3
compute_U_row(i,M,V,d,lambda1)
U[i,] = compute_U_row(i,M,V,d,lambda1)
print U

print "error 3"
print get_error(M,U,V)

i=0
compute_U_row(i,M,V,d,lambda1)
U[i,] = compute_U_row(i,M,V,d,lambda1)
print U

print "error 4"
print get_error(M,U,V)

i=1
compute_U_row(i,M,V,d,lambda1)
U[i,] = compute_U_row(i,M,V,d,lambda1)
print U

print "error 5"
print get_error(M,U,V)

[[ 0.77132064  0.02075195]
 [ 0.63364823  0.74880388]
 [ 0.49850701  0.22479665]
 [ 0.19806286  0.76053071]]
[[ 0.77132064  0.02075195  0.63364823  0.74880388  0.49850701]
 [ 0.22479665  0.19806286  0.76053071  0.16911084  0.08833981]]
error 0
3.07634148073
[[ 0.77132064  0.02075195]
 [ 1.87879054  0.77399723]
 [ 0.49850701  0.22479665]
 [ 0.19806286  0.76053071]]
error 1
3.00059486148
[[ 0.77132064  0.02075195]
 [ 1.87879054  0.77399723]
 [ 3.40360287  0.87884019]
 [ 0.19806286  0.76053071]]
error 2
2.55995959432
[[ 0.77132064  0.02075195]
 [ 1.87879054  0.77399723]
 [ 3.40360287  0.87884019]
 [ 2.12688876  1.24594828]]
error 3
2.33532726162
[[ 2.59946563  1.09288265]
 [ 1.87879054  0.77399723]
 [ 3.40360287  0.87884019]
 [ 2.12688876  1.24594828]]
error 4
1.94047964044
[[ 2.59946563  1.09288265]
 [ 1.87879054  0.77399723]
 [ 3.40360287  0.87884019]
 [ 2.12688876  1.24594828]]
error 5
1.94047964044


In [143]:

i = 1
seed = 10
d = 2
np.random.seed(seed)
U = np.zeros((M.shape[0],d),dtype=np.float)+np.random.rand(M.shape[0],d)
#U = np.zeros((M_rows,factors),dtype=np.float)+.2
print U
np.random.seed(seed)
V = np.zeros((d,M.shape[1]),dtype=np.float)+np.random.rand(d,M.shape[1])
print V
lambda2 = 1
print "error 0"
print get_error(M,U,V)
j = 0
V[:,j] = compute_V_col(j,M,U,d,lambda2)
print V
print "error 0"
print get_error(M,U,V)

j = 1
V[:,j] = compute_V_col(j,M,U,d,lambda2)
print V
print "error 1"
print get_error(M,U,V)

j = 2
V[:,j] = compute_V_col(j,M,U,d,lambda2)
print V
print "error 2"
print get_error(M,U,V)


j = 3
V[:,j] = compute_V_col(j,M,U,d,lambda2)
print V
print "error 3"
print get_error(M,U,V)

j = 4
V[:,j] = compute_V_col(j,M,U,d,lambda2)
print V
print "error 4"
print get_error(M,U,V)


i=1
compute_U_row(i,M,V,d,lambda1)
U[i,] = compute_U_row(i,M,V,d,lambda1)
print U

print "error 5"
print get_error(M,U,V)

[[ 0.77132064  0.02075195]
 [ 0.63364823  0.74880388]
 [ 0.49850701  0.22479665]
 [ 0.19806286  0.76053071]]
[[ 0.77132064  0.02075195  0.63364823  0.74880388  0.49850701]
 [ 0.22479665  0.19806286  0.76053071  0.16911084  0.08833981]]
error 0
3.07634148073
[[ 1.70931944  0.02075195  0.63364823  0.74880388  0.49850701]
 [ 0.88092837  0.19806286  0.76053071  0.16911084  0.08833981]]
error 0
3.00954930967
[[ 1.70931944  1.24166015  0.63364823  0.74880388  0.49850701]
 [ 0.88092837  1.05822645  0.76053071  0.16911084  0.08833981]]
error 1
2.89774619573
[[ 1.70931944  1.24166015  1.64592764  0.74880388  0.49850701]
 [ 0.88092837  1.05822645  1.3108199   0.16911084  0.08833981]]
error 2
2.8199075713
[[ 1.70931944  1.24166015  1.64592764  3.16680845  0.49850701]
 [ 0.88092837  1.05822645  1.3108199   2.21871135  0.08833981]]
error 3
2.1302162309
[[ 1.70931944  1.24166015  1.64592764  3.16680845  2.29185835]
 [ 0.88092837  1.05822645  1.3108199   2.21871135 -0.1739486 ]]
error 4
1.91950221781

In [177]:
i = 1
seed = 10
d = 2
np.random.seed(seed)
U = np.zeros((M.shape[0],d),dtype=np.float)+np.random.rand(M.shape[0],d)
#U = np.zeros((M_rows,factors),dtype=np.float)+.2
print U
np.random.seed(seed)
V = np.zeros((d,M.shape[1]),dtype=np.float)+np.random.rand(d,M.shape[1])
print V


[[ 0.77132064  0.02075195]
 [ 0.63364823  0.74880388]
 [ 0.49850701  0.22479665]
 [ 0.19806286  0.76053071]]
[[ 0.77132064  0.02075195  0.63364823  0.74880388  0.49850701]
 [ 0.22479665  0.19806286  0.76053071  0.16911084  0.08833981]]


In [178]:
compute_U_row(i,M,V,d,lambda1)
i = 0
j = 0
lambda1 = 1
lambda2 = 1
d = 2
error = []
error.append(get_error(M,U,V))
M_rows, M_cols = M.shape
for count in xrange(100):
    print count
    U_row_number = i%M_rows
    U[U_row_number,]=compute_U_row(U_row_number,M,V,d,lambda1)
    error.append(get_error(M,U,V))
    if error[-2] - error[-1] <= 0.000001:
        print error[-2] - error[-1]
        print "in if 1"
    #    break
    V_col_number = j%M_cols
    V[:,V_col_number]=compute_V_col(V_col_number,M,U,d,lambda2)
    error.append(get_error(M,U,V))
    if error[-2] - error[-1] <= 0.000001:
        print error[-2] - error[-1]
        print "in if 2"
    #    break
    i = i+1
    j = j+1

         

0
1
2
3
4
-0.034923434332
in if 1
5
-0.00948375274289
in if 1
6
-0.0798396318295
in if 1
-0.00480825906328
in if 2
7
-0.0205316846478
in if 1
-0.000272952545391
in if 2
8
9
-0.00114590864019
in if 1
10
-0.0354802687464
in if 1
11
-0.0162386323941
in if 1
12
-0.000253032245577
in if 1
13
-0.00324367406388
in if 1
14
15
-0.0115544006051
in if 1
16
-0.00931721425457
in if 1
17
-0.00757956557649
in if 1
18
-0.0066945473119
in if 1
19
-0.00876295484448
in if 1
20
-0.00826206995462
in if 1
21
-0.00654966543343
in if 1
22
-0.0226136813143
in if 1
23
-0.00234157290701
in if 1
24
25
26
-0.0199375193713
in if 1
27
-0.00396957258924
in if 1
28
29
30
-0.0142958136419
in if 1
31
-0.00316223341156
in if 1
32
33
34
-0.0249238812777
in if 1
-0.000976188756908
in if 2
35
-0.00615895582186
in if 1
36
-0.000187243604318
in if 2
37
-0.00580159954836
in if 1
38
-0.00766792512785
in if 1
39
-0.00420198844117
in if 1
-0.00343518793737
in if 2
40
41
-0.00391979096541
in if 1
42
-0.00940443609405
in if 1
43
-0

In [176]:
i
error

[3.0763414807340048,
 2.7884735469695379,
 2.787015837736174,
 2.7031740410966512,
 2.585750320148029,
 2.1107650496957886,
 2.068499509539834,
 1.8131169780114615,
 1.3493138769361499,
 1.3842373112681976]

In [179]:
print U
print V



[[ 1.03524443  1.99497238]
 [ 1.5178706   0.21031646]
 [ 2.23551385 -0.10421659]
 [ 1.45240245  0.8367609 ]]
[[ 1.82466561  0.99134598  1.11807603  2.14199026  0.57340465]
 [-0.3746096   0.43941905  0.84984238  0.77333748  1.75510065]]


In [180]:
M

array([[  1.,   2.,   3.,   4.,   5.],
       [ nan,   2.,  nan,   4.,   1.],
       [  5.,  nan,  nan,   5.,  nan],
       [ nan,   2.,   3.,   4.,  nan]])

In [181]:
np.dot(U,V)

array([[ 1.1416391 ,  1.90291428,  2.85289407,  3.76027041,  4.0949913 ],
       [ 2.69081971,  1.59715198,  1.87583057,  3.41390964,  1.23948061],
       [ 4.11810577,  2.17037292,  2.41090677,  4.70785429,  1.09894342],
       [ 2.33669013,  1.80752202,  2.33501124,  3.75813047,  2.30141392]])

## Stochastic Gradient Descent factorization

In [182]:
M

array([[  1.,   2.,   3.,   4.,   5.],
       [ nan,   2.,  nan,   4.,   1.],
       [  5.,  nan,  nan,   5.,  nan],
       [ nan,   2.,   3.,   4.,  nan]])

In [199]:
def sto_gra_U_row(i,M,U,V,d,lambda1,alpha):
    '''
     i = desired row number in U, which needs to be computed
     M = Utility matrix of size mXn
     V = V component of size dXn
     lambda1 = reg factor
    '''
    num_of_rows, num_of_cols = M.shape
    #print "num_of_users = {} and num_of_items = {}".format(num_of_rows, num_of_cols)
    
    #np.where will return a tuple.
    C = np.where(~np.isnan(M[i,]))[0]
    
    V_C = V[:,C].copy()
    M_i = M[i,:].copy()
    M_i = M_i[~np.isnan(M_i)]
    
    return alpha*(-1*np.dot((M_i - np.dot(U[i,],V_C)),V_C.T) + lambda1 * U[i,])
    
def sto_gra_V_col(j,M,U,V,d,lambda2,alpha):
    '''
     j = desired column number in V, which needs to be computed
     M = Utility matrix of size mXn
     U = U component of size mXd
     lambda2 = reg factor
    '''
    num_of_rows, num_of_cols = M.shape
    #print "num_of_users = {} and num_of_items = {}".format(num_of_rows, num_of_cols)
    
    #np.where will return a tuple.
    R = np.where(~np.isnan(M[:,j]))[0]
    
    U_R = U[R,:].copy()
    M_j = M[:,j].copy()
    M_j = M_j[~np.isnan(M_j)]
    return alpha*(-1*np.dot((M_j - np.dot(U_R,V[:,j])).T,U_R) + lambda2 * V[:,j])

In [203]:
i = 1
seed = 10
d = 2
np.random.seed(seed)
U = np.zeros((M.shape[0],d),dtype=np.float)+np.random.rand(M.shape[0],d)
#U = np.zeros((M_rows,factors),dtype=np.float)+.2
print U
np.random.seed(seed)
V = np.zeros((d,M.shape[1]),dtype=np.float)+np.random.rand(d,M.shape[1])
print V
lambda2 = 1
alpha = 0.001

[[ 0.77132064  0.02075195]
 [ 0.63364823  0.74880388]
 [ 0.49850701  0.22479665]
 [ 0.19806286  0.76053071]]
[[ 0.77132064  0.02075195  0.63364823  0.74880388  0.49850701]
 [ 0.22479665  0.19806286  0.76053071  0.16911084  0.08833981]]


In [204]:
sto_gra_V_col(i,M,U,V,d,lambda2,alpha)

array([-0.00303684, -0.0026231 ])

In [205]:
i = 1
j = 1
print get_error(M,U,V)

U[i,] = U[i,] - sto_gra_U_row(i,M,U,V,d,lambda1,alpha)
V[:,j] = V[:,j] - sto_gra_V_col(j,M,U,V,d,lambda2,alpha)
print get_error(M,U,V)

i = 2
j = 2

U[i,] = U[i,] - sto_gra_U_row(i,M,U,V,d,lambda1,alpha)
V[:,j] = V[:,j] - sto_gra_V_col(j,M,U,V,d,lambda2,alpha)
print get_error(M,U,V)

i = 3
j = 3

U[i,] = U[i,] - sto_gra_U_row(i,M,U,V,d,lambda1,alpha)
V[:,j] = V[:,j] - sto_gra_V_col(j,M,U,V,d,lambda2,alpha)
print get_error(M,U,V)




3.07634148073
3.07575462891
3.0744096541
3.07140888374


In [238]:
error=[]
print M
seed = 10
np.random.seed(seed)
U = np.zeros((M.shape[0],d),dtype=np.float)+np.random.rand(M.shape[0],d)
print U
np.random.seed(seed)
V = np.zeros((d,M.shape[1]),dtype=np.float)+np.random.rand(d,M.shape[1])
print V
lambda2 = 1
alpha = 0.01
for count in xrange(10000):
    pick_1 = np.random.random_integers(0,1)
    if pick_1 == 0:
        pick_2 = np.random.random_integers(0,M.shape[0]-1)
        #print "pick-2 for U {}".format(pick_2)
        #print sto_gra_U_row(pick_2,M,U,V,d,lambda1,alpha)
        U[pick_2,] = U[pick_2,] - sto_gra_U_row(pick_2,M,U,V,d,lambda1,alpha)
        error.append(get_error(M,U,V))
    else:
        pick_2 = np.random.random_integers(0,M.shape[1]-1)
        #print "pick-2 for V {}".format(pick_2)
        V[:,pick_2] = V[:,pick_2] - sto_gra_V_col(pick_2,M,U,V,d,lambda2,alpha)
        error.append(get_error(M,U,V))        
print U
print V
print error[-1:-5]

[[  1.   2.   3.   4.   5.]
 [ nan   2.  nan   4.   1.]
 [  5.  nan  nan   5.  nan]
 [ nan   2.   3.   4.  nan]]
[[ 0.77132064  0.02075195]
 [ 0.63364823  0.74880388]
 [ 0.49850701  0.22479665]
 [ 0.19806286  0.76053071]]
[[ 0.77132064  0.02075195  0.63364823  0.74880388  0.49850701]
 [ 0.22479665  0.19806286  0.76053071  0.16911084  0.08833981]]


C:\Users\a152700\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
C:\Users\a152700\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead
C:\Users\a152700\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:21: DeprecationWarning: This function is deprecated. Please call randint(0, 4 + 1) instead


[[ 2.22856092 -0.29254661]
 [ 1.04094687  1.12437657]
 [ 1.193682    1.89278508]
 [ 1.51593869  0.7143278 ]]
[[ 0.73695253  0.92758397  1.33667343  1.8598199   1.76743234]
 [ 1.7108638   0.56185891  0.43268559  1.3143674  -0.53408243]]
[]


In [239]:
np.dot(U,V)

array([[ 1.14183621,  1.90280747,  2.85227746,  3.76020823,  4.09507464],
       [ 2.6907836 ,  1.59730663,  1.87790757,  3.41381762,  1.23929339],
       [ 4.11798445,  2.17071846,  2.41454384,  4.70784854,  1.0988489 ],
       [ 2.33929244,  1.80751187,  2.33539432,  3.75826213,  2.29780914]])

In [240]:
error[-1]

0.47160139751294938

In [247]:
error

[3.0706433407402653,
 3.0576878181153817,
 3.0319650133161677,
 3.0185720682533077,
 3.0166102475997074,
 3.0145844278492691,
 3.0017206347674117,
 2.9997404112013433,
 2.9725453983223855,
 2.9672440734661514,
 2.9522028399529705,
 2.9499293138164711,
 2.9477241567815375,
 2.9420226274706702,
 2.934422100064277,
 2.9321595815309189,
 2.9304144657283864,
 2.9252419230024342,
 2.9103502073327645,
 2.9079360899651845,
 2.8928448997625966,
 2.8907874277245371,
 2.8846758441060456,
 2.8769300780118008,
 2.8712621802546696,
 2.8658360895897506,
 2.8499538514492189,
 2.8350146405435912,
 2.8285524525391645,
 2.8266922337749794,
 2.8110712645590139,
 2.7960115294940922,
 2.7812477344385691,
 2.7787310468356923,
 2.7757634281691708,
 2.7679579467490054,
 2.7540525491592951,
 2.7410148258260145,
 2.7258283978126174,
 2.7177943178375803,
 2.7096555706057419,
 2.6949348845778349,
 2.6874088265660783,
 2.680416379801156,
 2.6776206243916292,
 2.6749261204210493,
 2.6719011013077933,
 2.663497085973

In [242]:
np.dot(U,V)

array([[ 1.14183621,  1.90280747,  2.85227746,  3.76020823,  4.09507464],
       [ 2.6907836 ,  1.59730663,  1.87790757,  3.41381762,  1.23929339],
       [ 4.11798445,  2.17071846,  2.41454384,  4.70784854,  1.0988489 ],
       [ 2.33929244,  1.80751187,  2.33539432,  3.75826213,  2.29780914]])

In [243]:
M

array([[  1.,   2.,   3.,   4.,   5.],
       [ nan,   2.,  nan,   4.,   1.],
       [  5.,  nan,  nan,   5.,  nan],
       [ nan,   2.,   3.,   4.,  nan]])